In [1]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion
from sklearn.impute import MissingIndicator
from sklearn.pipeline import Pipeline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from os import path

In [2]:
from sklearn import tree, linear_model, neighbors 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

In [3]:
# prepare data for risk prediction
df = pd.read_csv("heloc_dataset_v1.csv")

In [4]:
df.head(10)

,RiskPerformance,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,...,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,Bad,55,144,4,84,20,3,0,83,2,...,43,0,0,0,33,-8,8,1,1,69
1,Bad,61,58,15,41,2,4,4,100,-7,...,67,0,0,0,0,-8,0,-8,-8,0
2,Bad,67,66,5,24,9,0,0,100,-7,...,44,0,4,4,53,66,4,2,1,86
3,Bad,66,169,1,73,28,1,1,93,76,...,57,0,5,4,72,83,6,4,3,91
4,Bad,81,333,27,132,12,0,0,100,-7,...,25,0,1,1,51,89,3,1,0,80
5,Bad,59,137,11,78,31,0,0,91,1,...,47,0,0,0,62,93,12,4,3,94
6,Good,54,88,7,37,25,0,0,92,9,...,58,0,4,4,89,76,7,7,2,100
7,Good,68,148,7,65,17,0,0,83,31,...,44,0,0,0,28,48,2,2,2,40
8,Bad,59,324,2,138,24,0,0,85,5,...,26,0,1,1,68,-8,7,1,3,90
9,Bad,61,79,4,36,19,0,0,95,5,...,26,0,6,6,31,86,5,3,1,62


In [5]:
df.shape

(10459, 24)

In [6]:
X = df.iloc[:,1:24]
Y = df.iloc[:,0]
Y = Y.replace(["Bad","Good"],[1,0])

In [7]:
X.shape

(10459, 23)

In [8]:
Y

0        1
1        1
2        1
3        1
4        1
        ..
10454    0
10455    1
10456    1
10457    1
10458    1
Name: RiskPerformance, Length: 10459, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(X, test_size=0.2, random_state=1234)
Y_train, Y_test = train_test_split(Y, test_size=0.2, random_state=1234)

In [10]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(8367, 23) (2092, 23) (8367,) (2092,)


In [11]:
X_train.head()

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
7175,80,198,19,94,16,0,0,100,-7,7,...,25,0,0,0,25,-8,3,1,2,40
8239,81,141,3,59,34,1,0,97,54,6,...,51,0,1,1,4,94,2,2,0,44
4675,69,410,1,124,49,0,0,100,-7,7,...,23,1,1,1,41,15,11,2,1,58
2600,68,264,5,72,54,2,2,96,30,6,...,46,-7,1,1,18,83,4,7,1,50
1061,58,254,9,142,31,2,2,82,25,6,...,67,0,5,5,61,33,2,4,1,88


In [12]:
# preprocessing
df_train = pd.concat([X_train, Y_train],axis = 1)

train_filter = X_train['ExternalRiskEstimate'] != -9
X_train = X_train[train_filter]
Y_train = Y_train[train_filter]

test_filter = X_test['ExternalRiskEstimate'] != -9
X_test = X_test[test_filter]
Y_test = Y_test[test_filter]

In [13]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(7907, 23) (1954, 23) (7907,) (1954,)


In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion
from sklearn.impute import MissingIndicator
from sklearn.pipeline import Pipeline

do_nothing_imputer = ColumnTransformer([("Imputer -7 to mean", SimpleImputer(missing_values=-7, strategy='mean'), [])], remainder='passthrough')

feature_expansion = FeatureUnion([("do nothing", do_nothing_imputer),
                                  ("add features for -7", MissingIndicator(missing_values=-7, features='missing-only')),
                                  ("add features for -8", MissingIndicator(missing_values=-8, features='missing-only'))])
 
pipeline = Pipeline([("expand features", feature_expansion), 
                 ("replace -7 with -8", SimpleImputer(missing_values=-7, strategy='constant', fill_value=-8)),
                 ("replace -8 with mean", SimpleImputer(missing_values=-8, strategy='mean'))])

arr_X_train_t = pipeline.fit_transform(X_train)
arr_X_train_t.shape

(7907, 34)

In [15]:
from sklearn.impute import MissingIndicator
minus_7_indicator_transformer = MissingIndicator(missing_values=-7, features='missing-only').fit(X_train)
minus_8_indicator_transformer = MissingIndicator(missing_values=-8, features='missing-only').fit(X_train)

col_names_minus_7 = X_train.columns.values[minus_7_indicator_transformer.features_].tolist() 
col_names_minus_7 = list(map(lambda s:str(s)+'=-7',col_names_minus_7))
col_names_minus_8 = X_train.columns.values[minus_8_indicator_transformer.features_].tolist() 
col_names_minus_8 = list(map(lambda s:str(s)+'=-8',col_names_minus_8))
column_names = X_train.columns.values.tolist() + col_names_minus_7 + col_names_minus_8

In [16]:
X_train_t = pd.DataFrame(arr_X_train_t, columns=column_names)

In [17]:
X_train_t.shape

(7907, 34)

In [18]:
X_test_t = pipeline.transform(X_test)
X_test_t = pd.DataFrame(X_test_t, columns = column_names)

In [19]:
X_train_t_tr, X_train_t_val = train_test_split(X_train_t, test_size=0.25, random_state=1234)
Y_train_t_tr, Y_train_t_val = train_test_split(Y_train, test_size=0.25, random_state=1234)

In [20]:
print('Train data:      X_train_t_tr:', X_train_t_tr.shape, ' Y_train_t_tr:', Y_train_t_tr.shape)
print('Validation data: X_train_t_val:', X_train_t_val.shape, 'Y_train_t_val:', Y_train_t_val.shape)
print('Test data:       X_test_t:', X_test_t.shape, '     Y_test:', Y_test.shape)


Train data:      X_train_t_tr: (5930, 34)  Y_train_t_tr: (5930,)
Validation data: X_train_t_val: (1977, 34) Y_train_t_val: (1977,)
Test data:       X_test_t: (1954, 34)      Y_test: (1954,)


In [21]:
# model training
log_reg = LogisticRegression(max_iter=10000, random_state=0).fit(X_train_t_tr, Y_train_t_tr) # Logistic regression
svc     = SGDClassifier(max_iter=1000, random_state=0).fit(X_train_t_tr, Y_train_t_tr)       # Linear SVC
nb      = BernoulliNB().fit(X_train_t_tr, Y_train_t_tr)                                      # Naive Bayes
lda     = LinearDiscriminantAnalysis().fit(X_train_t_tr, Y_train_t_tr)                       # LDA
tree    = tree.DecisionTreeClassifier().fit(X_train_t_tr, Y_train_t_tr)                      # Desicion Tree
knn     = neighbors.KNeighborsClassifier().fit(X_train_t_tr, Y_train_t_tr)                   # KNN

In [22]:
print('Train accuracy')
print('Log. Reg. accuracy: %.3f'%log_reg.score(X_train_t_tr, Y_train_t_tr))
print('Linear SVC accuracy: %.3f'%svc.score(X_train_t_tr, Y_train_t_tr))
print('Naive Bayes accuracy: %.3f'%nb.score(X_train_t_tr, Y_train_t_tr))
print('LDA accuracy: %.3f'%lda.score(X_train_t_tr, Y_train_t_tr)) 
print('Tree accuracy: %.3f'%tree.score(X_train_t_tr, Y_train_t_tr))
print('KNN accuracy: %.3f'%knn.score(X_train_t_tr, Y_train_t_tr))

print('\nValidation accuracy')
print('Log. Reg. accuracy: %.5f'%log_reg.score(X_train_t_val, Y_train_t_val))
print('Linear SVC accuracy: %.5f'%svc.score(X_train_t_val, Y_train_t_val))
print('Naive Bayes accuracy: %.5f'%nb.score(X_train_t_val, Y_train_t_val))
print('LDA accuracy: %.5f'%lda.score(X_train_t_val, Y_train_t_val)) 
print('Tree accuracy: %.5f'%tree.score(X_train_t_val, Y_train_t_val))
print('KNN accuracy: %.5f'%knn.score(X_train_t_val, Y_train_t_val))

print('\nTest accuracy')
print('Log. Reg. accuracy: %.5f'%log_reg.score(X_test_t,Y_test))
print('Linear SVC accuracy: %.5f'%svc.score(X_test_t,Y_test))
print('Naive Bayes accuracy: %.5f'%nb.score(X_test_t,Y_test))
print('LDA accuracy: %.5f'%lda.score(X_test_t,Y_test))
print('Tree accuracy: %.5f'%tree.score(X_test_t,Y_test))
print('KNN accuracy: %.5f'%knn.score(X_test_t,Y_test))

Train accuracy
Log. Reg. accuracy: 0.741
Linear SVC accuracy: 0.728
Naive Bayes accuracy: 0.676
LDA accuracy: 0.741
Tree accuracy: 1.000
KNN accuracy: 0.775

Validation accuracy
Log. Reg. accuracy: 0.74406
Linear SVC accuracy: 0.72939
Naive Bayes accuracy: 0.67881
LDA accuracy: 0.74355
Tree accuracy: 0.63784
KNN accuracy: 0.67931

Test accuracy
Log. Reg. accuracy: 0.72211
Linear SVC accuracy: 0.70522
Naive Bayes accuracy: 0.65302
LDA accuracy: 0.72723
Tree accuracy: 0.63153
KNN accuracy: 0.67349


In [23]:
# processing time
import time

start_time = time.time()
log_reg.fit(X_test_t,Y_test)
end_time = time.time()
print("Logistic Regression Processing Time:", end_time - start_time)

start_time = time.time()
svc.fit(X_test_t,Y_test)
end_time = time.time()
print("Linear SVC Processing Time:", end_time - start_time)

start_time = time.time()
nb.fit(X_test_t,Y_test)
end_time = time.time()
print("Naive Bayes Processing Time:", end_time - start_time)

start_time = time.time()
lda.fit(X_test_t,Y_test)
end_time = time.time()
print("LDA Processing Time:", end_time - start_time)

start_time = time.time()
tree.fit(X_test_t,Y_test)
end_time = time.time()
print("Decision Tree Processing Time:", end_time - start_time)

start_time = time.time()
knn.fit(X_test_t,Y_test)
end_time = time.time()
print("KNN Processing Time:", end_time - start_time)

Logistic Regression Processing Time: 1.7161731719970703
Linear SVC Processing Time: 0.03702974319458008
Naive Bayes Processing Time: 0.005909442901611328
LDA Processing Time: 0.020890235900878906
Decision Tree Processing Time: 0.03191041946411133
KNN Processing Time: 0.003125429153442383


In [24]:
# modify the feature number
from sklearn.feature_selection import SelectKBest, f_classif
selector = SelectKBest(f_classif, k=32) # 32 features have the highest accuracy
X_train_t_tr = selector.fit_transform(X_train_t_tr, Y_train_t_tr)
X_test_t = selector.transform(X_test_t)
log_reg = LogisticRegression(max_iter=10000, random_state=0).fit(X_train_t_tr, Y_train_t_tr)

In [25]:
selected_feature_indices = selector.get_support()
selected_feature_names = X_train_t.columns[selected_feature_indices]
selected_feature_names

Index(['ExternalRiskEstimate', 'MSinceOldestTradeOpen',
       'MSinceMostRecentTradeOpen', 'AverageMInFile', 'NumSatisfactoryTrades',
       'NumTrades60Ever2DerogPubRec', 'NumTrades90Ever2DerogPubRec',
       'PercentTradesNeverDelq', 'MSinceMostRecentDelq',
       'MaxDelq2PublicRecLast12M', 'MaxDelqEver', 'NumTotalTrades',
       'NumTradesOpeninLast12M', 'PercentInstallTrades',
       'MSinceMostRecentInqexcl7days', 'NumInqLast6M', 'NumInqLast6Mexcl7days',
       'NetFractionRevolvingBurden', 'NetFractionInstallBurden',
       'NumRevolvingTradesWBalance', 'NumInstallTradesWBalance',
       'NumBank2NatlTradesWHighUtilization', 'PercentTradesWBalance',
       'MSinceMostRecentDelq=-7', 'MSinceMostRecentInqexcl7days=-7',
       'MSinceMostRecentDelq=-8', 'MSinceMostRecentInqexcl7days=-8',
       'NetFractionRevolvingBurden=-8', 'NetFractionInstallBurden=-8',
       'NumRevolvingTradesWBalance=-8', 'NumInstallTradesWBalance=-8',
       'NumBank2NatlTradesWHighUtilization=-8'],
     

In [26]:
print("Training Accuracy:",log_reg.score(X_train_t_tr, Y_train_t_tr))
X_train_t_val = selector.transform(X_train_t_val)
Y_pred_t_val = log_reg.predict(X_train_t_val)
val_accuracy = accuracy_score(Y_train_t_val, Y_pred_t_val)
print("Validation Accuracy:", val_accuracy)
Y_pred_t = log_reg.predict(X_test_t)
test_accuracy = accuracy_score(Y_test, Y_pred_t)
print("Test Accuracy:", test_accuracy)

start_time = time.time()
log_reg.fit(X_test_t,Y_test)
end_time = time.time()
print("Logistic Regression Processing Time:", end_time - start_time)

Training Accuracy: 0.7433389544688027
Validation Accuracy: 0.7440566514921598
Test Accuracy: 0.7231320368474923
Logistic Regression Processing Time: 2.4898977279663086


In [27]:
log_reg

LogisticRegression(max_iter=10000, random_state=0)

In [28]:
import pickle
with open('log_reg_model.pkl', 'wb') as file:
    pickle.dump(log_reg, file)